In [1]:
import os
import pandas as pd

In [3]:
# Specify your base folder path (e.g., "data" folder)
BASE_PATH = '/home/blacktea/Downloads/Tung/Study/paper/OSA and Osteoporosis'
INPUT_PATH = f'{BASE_PATH}/Data Set/csv'

# Create a dictionary to store all DataFrames
dataframes = {}

# Get all CSV files in the folder and sort them
file_list = sorted([f for f in os.listdir(INPUT_PATH) if f.endswith(".csv")])

# Read each CSV file in the sorted order
for file_name in file_list:
    file_path = os.path.join(INPUT_PATH, file_name)
    dataframes[file_name] = pd.read_csv(file_path)

# Display file names to confirm successful loading
for file_name, df in dataframes.items():
    print(f"File name: {file_name}")

File name: 2017-2022-Meger.csv
File name: 2017年呼吸中止症報表.csv
File name: 2017年骨密報表.csv
File name: 2018年呼吸中止症報表.csv
File name: 2018年骨密報表.csv
File name: 2019年呼吸中止症報表.csv
File name: 2019年骨密報表.csv
File name: 2020年呼吸中止症報表.csv
File name: 2020年骨密報表.csv
File name: 2021年呼吸中止症報表.csv
File name: 2021年骨密報表.csv
File name: 2022年呼吸中止症報表.csv
File name: 2022年骨密報表.csv


In [5]:
OUTPUT_PATH = f'{BASE_PATH}/Data Set/filter'
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Specify the base file name and base feature name
base_file_name = '2017-2022-Meger.csv'  # Replace with the base file name
base_key_feature = 'id_number'  # Replace with the feature name in the base file

# Specify the mapping of feature names for other files, format: {"file_name": "feature_name"}
other_key_features = {
    '2017年呼吸中止症報表.csv': '去個資編號',
    '2018年呼吸中止症報表.csv': '去個資編號',
    '2019年呼吸中止症報表.csv': '去個資編號',
    '2020年呼吸中止症報表.csv': '去個資編號',
    '2021年呼吸中止症報表.csv': '去個資編號',
    '2022年呼吸中止症報表.csv': '去個資編號',

    '2017年骨密報表.csv': '去個資編號',
    '2018年骨密報表.csv': '去個資編號',
    '2019年骨密報表.csv': '去個資編號',
    '2020年骨密報表.csv': '去個資編號',
    '2021年骨密報表.csv': '去個資編號',
    '2022年骨密報表.csv': '去個資編號'
}

# Assuming 'dataframes' contains all uploaded files as a dictionary
# Ensure the base file exists in the uploaded files
if base_file_name in dataframes:
    # Extract the base DataFrame
    base_df = dataframes[base_file_name]

    # Create a variable to store the filtered base data
    selected_rows = pd.DataFrame(columns=base_df.columns)

    # Initialize counters
    osa_sample_count = 0  # Sample count from OSA files
    bmd_sample_count = 0  # Sample count from BMD files

    # Filter the base DataFrame based on the feature values from other files
    for other_file, feature_name in other_key_features.items():
        if other_file in dataframes:
            # Extract the feature values from the other file
            other_values = set(dataframes[other_file][feature_name])

            # Filter the base DataFrame for matching rows and avoid duplicates
            matching_rows = base_df[base_df[base_key_feature].isin(other_values) & ~base_df[base_key_feature].isin(selected_rows[base_key_feature])]
            selected_rows = pd.concat([selected_rows, matching_rows], ignore_index=True)

            # Count the samples from OSA and BMD files
            if '呼吸中止症' in other_file:
                osa_sample_count += len(matching_rows)
            elif '骨密' in other_file:
                bmd_sample_count += len(matching_rows)

            print(f"Found {len(matching_rows)} matching rows in {other_file}")

    # Remove duplicate rows (if any) based on 'id_number' after all filtering
    selected_rows.drop_duplicates(subset=base_key_feature, inplace=True)

    # Display the sample count after filtering
    print(f"Total samples after filtering: {len(selected_rows)}")
    print(f"Samples filtered from OSA reports: {osa_sample_count}")
    print(f"Samples filtered from BMD reports: {bmd_sample_count}")

    # Check if there are any duplicate 'id_number's
    duplicate_ids = selected_rows[selected_rows.duplicated(subset=base_key_feature, keep=False)]
    print(f"There are {len(duplicate_ids)} rows with duplicate 'id_number'")

    # Save the filtered base file as a new CSV file
    selected_rows.to_csv(f'{OUTPUT_PATH}/2017-2022-filter.csv', index=False)
    print("Filtered base file saved as '2017-2022-search.csv'")

else:
    print(f"Base file '{base_file_name}' not found. Please check the file name.")

Found 341 matching rows in 2017年呼吸中止症報表.csv
Found 197 matching rows in 2018年呼吸中止症報表.csv
Found 134 matching rows in 2019年呼吸中止症報表.csv
Found 145 matching rows in 2020年呼吸中止症報表.csv
Found 123 matching rows in 2021年呼吸中止症報表.csv
Found 92 matching rows in 2022年呼吸中止症報表.csv
Found 901 matching rows in 2017年骨密報表.csv
Found 705 matching rows in 2018年骨密報表.csv
Found 693 matching rows in 2019年骨密報表.csv
Found 714 matching rows in 2020年骨密報表.csv
Found 732 matching rows in 2021年骨密報表.csv
Found 703 matching rows in 2022年骨密報表.csv
Total samples after filtering: 5480
Samples filtered from OSA reports: 1032
Samples filtered from BMD reports: 4448
There are 0 rows with duplicate 'id_number'
Filtered base file saved as '2017-2022-search.csv'


/tmp/ipykernel_1077660/1721184110.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  selected_rows = pd.concat([selected_rows, matching_rows], ignore_index=True)


In [3]:
# Assuming 'dataframes' contains all uploaded files as a dictionary
# Ensure the base file exists in the uploaded files
if base_file_name in dataframes:
    # Extract the base DataFrame
    base_df = dataframes[base_file_name]

    # Initialize counters
    osa_ids = set()  # Used to store IDs selected by OSA (Sleep Apnea) reports
    bmd_ids = set()  # Used to store IDs selected by BMD (Bone Mineral Density) reports

    # Filter the base DataFrame based on the feature values from other files
    for other_file, feature_name in other_key_features.items():
        if other_file in dataframes:
            # Extract the feature values from the other file
            other_values = set(dataframes[other_file][feature_name])

            # Check if it's an OSA or BMD file
            if '呼吸中止症' in other_file:
                osa_ids.update(other_values)  # Update the IDs selected by OSA reports
            elif '骨密' in other_file:
                bmd_ids.update(other_values)  # Update the IDs selected by BMD reports

    # Select the rows from the base DataFrame that are in the OSA-selected IDs
    selected_osa = base_df[base_df[base_key_feature].isin(osa_ids)]
    # Select the rows from the base DataFrame that are in the BMD-selected IDs
    selected_bmd = base_df[base_df[base_key_feature].isin(bmd_ids)]

    # Select the rows that are in both OSA and BMD selected IDs
    selected_common = base_df[base_df[base_key_feature].isin(osa_ids.intersection(bmd_ids))]

    # Calculate the number of samples selected by only OSA or only BMD
    only_osa = selected_osa[~selected_osa[base_key_feature].isin(selected_bmd[base_key_feature])]
    only_bmd = selected_bmd[~selected_bmd[base_key_feature].isin(selected_osa[base_key_feature])]

    # Display results
    print(f"Samples selected from OSA reports: {len(selected_osa)}")
    print(f"Samples selected from BMD reports: {len(selected_bmd)}")
    print(f"Samples selected from OSA reports but not from BMD reports: {len(only_osa)}")
    print(f"Samples selected from BMD reports but not from OSA reports: {len(only_bmd)}")
    print(f"Samples selected by both OSA and BMD reports: {len(selected_common)}")
else:
    print(f"Base file '{base_file_name}' not found. Please check the file name.")

Samples selected from OSA reports: 1032
Samples selected from BMD reports: 4493
Samples selected from OSA reports but not from BMD reports: 987
Samples selected from BMD reports but not from OSA reports: 4448
Samples selected by both OSA and BMD reports: 45


In [6]:
# Read the CSV file into a DataFrame
INPUT_PATH = f'{BASE_PATH}/Data Set/filter'
df = pd.read_csv(f'{INPUT_PATH}/2017-2022-filter.csv')  # Load the CSV data into a pandas DataFrame

In [7]:
# Find duplicate 'id_number' values
duplicate_counts = df['id_number'].value_counts()  # Count the occurrences of each 'id_number'

# Filter out the 'id_number' values that appear more than once
duplicates = duplicate_counts[duplicate_counts > 1]  # Get 'id_number' values with more than one occurrence

# Output the duplicate 'id_number' values and their count
if not duplicates.empty:  # Check if there are any duplicates
    print("The following 'id_number' values are duplicated:")
    print(duplicates)  # Print the duplicated 'id_number' and their counts
else:
    print("There are no duplicated 'id_number' values.")  # Print message if no duplicates found

There are no duplicated 'id_number' values.
